In [8]:
import sys
sys.path.append('/Users/lei/home/studyhall/smart-earth-sensing/office-demo-project/lib') 
 
import numpy as np
import librosa 
import scipy 

import torch 
 
from h5reader import *
from preprocessing import *
from python_speech_features import mfcc
from cnn import CNN 

In [17]:
raw, _, _ = read_data('/Users/lei/home/studyhall/smart-earth-sensing/office-demo-project/data/corpus/NHS__200m_4m_1_1000Hz_1000Hz_UTC8_202308240934.h5', [20], [21])

100%|██████████| 1/1 [00:00<00:00, 84.53it/s]


In [25]:
xs = np.array(list((divide_chunks(raw, 500))))
np.shape(xs) 

(120, 500, 1)

In [27]:
b, a = scipy.signal.butter(8, 0.6)
xs = [scipy.signal.filtfilt(b, a, [e[0] for e in s]) for s in xs]
print(np.shape(xs)) 
xs = [librosa.resample(np.array(s), orig_sr=1000, target_sr=800) for s in tqdm(xs)]
print(np.shape(xs)) 
xs = [[np.swapaxes(librosa.feature.mfcc(y=s, sr=800, n_mfcc=61, hop_length=16), 0, 1)] for s in tqdm(xs)]
print(np.shape(xs)) 

(120, 1, 61)


100%|██████████| 120/120 [00:00<00:00, 8209.64it/s]


(120, 1, 49)


  0%|          | 0/120 [00:00<?, ?it/s]/Users/lei/anaconda3/envs/smart-earth-sensing/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=49
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 319.77it/s]

(120, 1, 61, 1, 4)


In [14]:
xs = torch.tensor(xs).to(torch.float32)
xs.size() 

/var/folders/0v/r030dv3s4tn1pl2pq7k7ywcw0000gn/T/ipykernel_72129/934672053.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xs = torch.tensor(xs).to(torch.float32)


torch.Size([120, 1, 13, 49])

In [9]:
model = CNN(torch.device("mps"))
model.load_state_dict(torch.load('../models/cnn_100e_0005lr_500hz.pt')) 

<All keys matched successfully>

In [12]:
ys = model()
ys 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (120x5120 and 6656x64)